In [71]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import os
from torchvision.datasets import ImageFolder
import shutil

In [78]:
train_dir = "D:\Study\Comp 6321 ML\Project\Dataset 1\Dataset 1\Colorectal Cancer\Train"
test_dir = "D:\Study\Comp 6321 ML\Project\Dataset 1\Dataset 1\Colorectal Cancer\Test"

In [79]:
# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [80]:
train_dataset = ImageFolder(train_dir, transform=data_transforms['train'])
test_dataset = ImageFolder(test_dir, transform=data_transforms['test'])

In [81]:
# Create DataLoader for train and test
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [83]:
# Define the model - Example using a pre-trained ResNet18
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

In [84]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [85]:
# Train the model
def train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}')

In [86]:
# Evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')

In [87]:
train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=10)

Epoch [1/10] - Loss: 0.3078
Epoch [2/10] - Loss: 0.0715
Epoch [3/10] - Loss: 0.0510
Epoch [4/10] - Loss: 0.0233
Epoch [5/10] - Loss: 0.0176
Epoch [6/10] - Loss: 0.0114
Epoch [7/10] - Loss: 0.0125
Epoch [8/10] - Loss: 0.0137
Epoch [9/10] - Loss: 0.0070
Epoch [10/10] - Loss: 0.0065


In [88]:
evaluate_model(model, test_loader)

Test Accuracy: 0.9909
